In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
try:
    from amplpy import AMPL, tools
except ImportError:
    # Caso ocorra um erro ao importar, instala amplpy e tenta importar novamente
    %pip install amplpy
    from amplpy import AMPL, tools

In [3]:
licenca ="default"

In [4]:

ampl = tools.ampl_notebook(
    modules=["highs", "gokestrel", "gurobi", "cplex"],  # Módulos a serem instalados
    license_uuid=licenca,  # Licença a ser utilizada
    g=globals()  # Instancia o objeto AMPL e registra os comandos mágicos
)


# 1- Base

Importando a base pré-processada com o limite de produção diário de:
<ul>
    <li>120.000 peças para o “Tipo de buffer” CX e </li>
    <li>100.000 peças para o “Tipo de buffer” EN.</li>
 </ul>

In [5]:
df_process = pd.read_excel('../../01-Data/ordens_pre_process_revisado.xlsx')

df_process

,Unnamed: 0,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Tipo_de_buffer,Item_pai,Cor,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,CX,1865,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,CX,1865,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,CX,1865,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,CX,2489,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,CX,2988,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,EN,1863,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,EN,1863,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,EN,1863,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,EN,713,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707


In [6]:
df_process = df_process.rename(columns={'Unnamed: 0': "Id"})

In [7]:
df_process = df_process.rename(columns={'Capacidade': "Capacidade_rota"})

In [8]:
df_process.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit'],
      dtype='object')

In [9]:
df_process['tempo_espera_dias'] = (df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16')

In [10]:
# Definir a função para calcular a prioridade
def calcular_prioridade(delta_t):
    diferenca = delta_t_maximo - delta_t
    if diferenca <= 5:
        return delta_t_maximo
    elif diferenca <= 10:
        return delta_t_maximo - 10
    else:
        return delta_t_maximo - 20

In [11]:
delta_t_maximo = df_process['tempo_espera_dias'].max()
df_process['prioridade'] = df_process['tempo_espera_dias'].apply(calcular_prioridade)
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


In [12]:
(df_process['Data_da_ordem_de_produção'].max() - df_process['Data_da_ordem_de_produção']).dt.days.astype('int16').head(300)

0      98
1      98
2      98
3      37
4      37
       ..
295    20
296    20
297    20
298    20
299    20
Name: Data_da_ordem_de_produção, Length: 300, dtype: int16

In [13]:
df_process.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [14]:
df_process

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Cor,Tamanho,Inner,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,6.0,27,1,1,inf,1865_6.0,26_CX_5,3092300,98,98
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,6.0,21,1,1,inf,1865_6.0,26_CX_5,3321500,98,98
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,6.0,4,1,1,inf,1865_6.0,26_CX_5,3537200,98,98
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,674.0,179,1,3,inf,2489_674.0,116_CX_6,233624,37,78
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,451.0,109,1,1,inf,2988_451.0,16_CX_2,775500,37,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,2.0,38,1,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,2.0,58,1,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,2.0,16,1,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,275.0,16,1,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78


## Caixa padrão

In [15]:
df_caixas_tipo = pd.read_excel('../../01-Data/Dados - desafio disciplina RPVMM_edit.xlsx', sheet_name='Tipos de caixa')
df_caixas_tipo

,Tipo,Tipo de buffer,Comprimento,Largura,Altura,Fator de ocupação,Peças max,Peso max (kg)
0,Caixa 1,CX,380,285,285,0.9,170,23
1,Caixa 2,CX,586,436,272,1.2,170,23
2,Caixa 3,CX,728,427,270,0.9,170,23
3,Caixa 1,EN,380,285,285,0.9,512,23
4,Caixa 2,EN,586,436,272,1.2,512,23
5,Caixa 3,EN,728,427,270,0.9,512,23


In [16]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')

In [17]:
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']

df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [18]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns=['Tipo','volume_util_caixa'], values=['Peças_max', 'volume_util_caixa']).reset_index()

Tipo_de_buffer  Peças_max                        \
Tipo                                Caixa 1    Caixa 2    Caixa 3   
volume_util_caixa                27778950.0 83393894.4 75538008.0   
0                             CX      170.0      170.0      170.0   
1                             EN      512.0      512.0      512.0   

                  volume_util_caixa                          
Tipo                        Caixa 1     Caixa 2     Caixa 3  
volume_util_caixa        27778950.0  83393894.4  75538008.0  
0                        27778950.0  83393894.4  75538008.0  
1                        27778950.0  83393894.4  75538008.0

In [19]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [20]:
df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='Peças_max').add_prefix('Peças_max_').reset_index()

Tipo,Tipo_de_buffer,Peças_max_Caixa 1,Peças_max_Caixa 2,Peças_max_Caixa 3
0,CX,170,170,170
1,EN,512,512,512


In [21]:
df_caixas_tipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tipo               6 non-null      object 
 1   Tipo_de_buffer     6 non-null      object 
 2   Comprimento        6 non-null      int64  
 3   Largura            6 non-null      int64  
 4   Altura             6 non-null      int64  
 5   Fator_de_ocupação  6 non-null      float64
 6   Peças_max          6 non-null      int64  
 7   Peso_max_(kg)      6 non-null      int64  
 8   volume_util_caixa  6 non-null      float64
dtypes: float64(2), int64(5), object(2)
memory usage: 564.0+ bytes


In [22]:
df_caixas_tipo.columns

Index(['Tipo', 'Tipo_de_buffer', 'Comprimento', 'Largura', 'Altura',
       'Fator_de_ocupação', 'Peças_max', 'Peso_max_(kg)', 'volume_util_caixa'],
      dtype='object')

Temos um tipo de caixa padrão para cada tipo de buffer

In [23]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


Levar a capacidade das caixas para cada ordem de produção:

O **número de peças máximo** varia com o buffer. Já o *volume* é igual para ambos os tipos, mudando apenas para o **tipo de caixa**

In [24]:
df_caixa_volume_max= df_caixas_tipo.pivot(index='Tipo_de_buffer', columns='Tipo', values='volume_util_caixa').add_prefix('volume_util_caixa_').reset_index()

df_caixa_volume_max

Tipo,Tipo_de_buffer,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,CX,27778950.0,83393894.4,75538008.0
1,EN,27778950.0,83393894.4,75538008.0


In [25]:
df_process_capacitado = df_process.merge(df_caixa_volume_max, on='Tipo_de_buffer', how='inner')

df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,Rota,Capacidade_rota,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa 1,volume_util_caixa_Caixa 2,volume_util_caixa_Caixa 3
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1,inf,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,3,inf,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,1,inf,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,15,4.000000e+10,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,15,4.000000e+10,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0


In [26]:
df_process_capacitado.columns = df_process_capacitado.columns.str.replace(' ', '_')

In [27]:
import numpy as np
df_process_capacitado['volume_util_padrao'] = np.where(df_process_capacitado['Tipo_de_buffer'] == "EN", df_process_capacitado['volume_util_caixa_Caixa_3'], df_process_capacitado['volume_util_caixa_Caixa_2'])

In [28]:
df_process_capacitado = df_process_capacitado.merge(df_caixas_padrao[['Tipo_de_buffer', 'Peças_max']], on='Tipo_de_buffer', how='inner')
df_process_capacitado

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
0,0,4522,2023-08-16,633674988,428,289,25,0.560,2,CARTON_M,...,1865_6.0,26_CX_5,3092300,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
1,1,7928,2023-08-16,633675008,455,292,25,0.600,3,CARTON_M,...,1865_6.0,26_CX_5,3321500,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
2,2,9224,2023-08-16,633674996,478,296,25,0.580,1,CARTON_M,...,1865_6.0,26_CX_5,3537200,98,98,27778950.0,83393894.4,75538008.0,83393894.4,170
3,3,9002,2023-10-16,804068334,116,106,19,0.082,1,CARTON_S,...,2489_674.0,116_CX_6,233624,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
4,4,11552,2023-10-16,825051881,235,110,30,0.092,10,CARTON_M,...,2988_451.0,16_CX_2,775500,37,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62226,62226,6095,2023-11-22,623412941,420,278,28,0.213,2,CARTON_M,...,1863_2.0,48_EN_2,3269280,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62227,62227,5591,2023-11-22,623412950,427,305,31,0.233,1,CARTON_M,...,1863_2.0,48_EN_2,4037285,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62228,62228,11858,2023-11-22,623412933,408,264,30,0.195,2,CARTON_M,...,1863_2.0,48_EN_2,3231360,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
62229,62229,10583,2023-11-22,622808371,371,233,49,0.323,1,CARTON_M,...,713_275.0,48_EN_2,4235707,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [29]:
df_process_capacitado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62231 entries, 0 to 62230
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Id                         62231 non-null  int64         
 1   Ordem_de_produção          62231 non-null  int64         
 2   Data_da_ordem_de_produção  62231 non-null  datetime64[ns]
 3   Produto                    62231 non-null  int64         
 4   Comprimento_unit           62231 non-null  int64         
 5   Largura_unit               62231 non-null  int64         
 6   Altura_unit                62231 non-null  int64         
 7   Peso_unit                  62231 non-null  float64       
 8   Peças                      62231 non-null  int64         
 9   Caixa_padrão               62231 non-null  object        
 10  Loja                       62231 non-null  int64         
 11  Classe_de_onda             62231 non-null  int64         
 12  Tipo

In [30]:
df_process_capacitado['chave_loja_buffer_onda'].nunique()

1508

In [31]:
df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)

chave_loja_buffer_onda
10_EN_3    3459
28_EN_4    3187
16_EN_3    2093
2_EN_3     2086
31_EN_3    1802
33_EN_3    1786
8_EN_4     1620
10_EN_4    1509
9_EN_4     1464
3_EN_3     1403
25_EN_4    1367
18_EN_4    1360
48_EN_4    1355
8_EN_3     1348
10_EN_1    1329
23_EN_3    1299
11_EN_4    1228
3_EN_4     1162
2_EN_4     1156
9_EN_3     1127
Name: Peças, dtype: int64

In [32]:
(df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(20)).index

Index(['10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3',
       '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4',
       '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4',
       '2_EN_4', '9_EN_3'],
      dtype='object', name='chave_loja_buffer_onda')

In [33]:
# Selecionar as n lojas maiores de um único buffer
n = 3
lojas_selecionadas_maiores = list((df_process_capacitado.query('Tipo_de_buffer == "EN"').groupby('chave_loja_buffer_onda')['Peças'].sum().sort_values(ascending=False).head(n)).index)
lojas_selecionadas_maiores

['10_EN_3', '28_EN_4', '16_EN_3']

In [34]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
33800,33800,40,2023-10-30,847254655,443,233,32,0.303,10,CARTON_M,...,40_28.0,28_EN_4,3303008,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33801,33801,915,2023-10-30,847254621,445,242,33,0.323,10,CARTON_M,...,40_28.0,28_EN_4,3553770,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33802,33802,4051,2023-10-30,847254663,431,225,34,0.295,6,CARTON_M,...,40_28.0,28_EN_4,3297150,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33803,33803,8438,2023-10-30,847254639,432,248,32,0.310,12,CARTON_M,...,40_28.0,28_EN_4,3428352,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
33804,33804,8788,2023-10-30,847254680,445,244,33,0.328,5,CARTON_M,...,40_28.0,28_EN_4,3583140,23,78,27778950.0,83393894.4,75538008.0,75538008.0,512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [35]:
df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")['Peças'].sum()

8739

# 2- Modelagem

## 2.1- Múltiplas mochilas

Vamos adicionar uma condição em que o tipo de buffer varia conforme a chave da loja

<p>O peso máximo é mesmo, indenpendente o tipo de buffer. Além disso, nesse momento iremos utilizar o volume padrão para cada buffer</p>

Modelagem matemática 

Dado que temos $n$ tipos de itens $i$ para empacotar, para $k$ lojas, cada um com peso $w_{i k}$, um volume $v_{i k}$, um múltiplo $inner_{i k}$ da quantidade de itens a serem selecionados uma demanda $d_{i k}$, para todo $i \in \{1,...,n\}$ e $k \in \{1,...,r\}$, e um número $m$ de mochilas (caixas) diferentes com capacidade volumétrica $cv_{k}$, uma capacidade de peso $cp$ e uma capacidade em número de peças $np_{k}$, além disso, cada item $i$ da loja $k$ tem um tempo em espera $T_{i k}$ no backlog de produção e os itens com tempo de espera maior devem ser alocados na frente, então temos:

$$
\begin{aligned}
\text{maximizar} \quad &\sum_{i = 1}^{n} \sum_{k = 1}^{r} \sum_{j = 1}^{m}  \xi_{i j k} *inner_{i k}* T_{i k}\\
\text{sujeito a} \quad &\sum_{i = 1}^{n} w_{i k} \xi_{i j k}*inner_{i k} \leq cp*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\}\\
&\sum_{i = 1}^{n} v_{i k} \xi_{i j k} *inner_{i k} \leq cv_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{i = 1}^{n} \xi_{i j k} *inner_{i k} \leq np_{k}*y_{j k} \quad \forall j \in \{1,...,m\} , k \in \{1,...,r\} \\
&\sum_{j = 1}^{m} \xi_{i j k} *inner_{i k} \leq  d_{i k} \quad \forall i \in \{1,...,n\}, k \in \{1,...,r\}\\

&\xi_{i j k} \geq0 \in \mathbb{Z} \quad \forall i \in \{1,...,n\}, j \in \{1,...,m\} , k \in \{1,...,r\} \\
&y_{j k} \in \{0,1\} \quad \forall j \in \{1,...,m\}, k \in \{1,...,r\}
\end{aligned}
$$


 Seja $\quad$ $\xi_{i j k}$= número de itens tipo $i$ adicionados na caixa $j$, para a loja $k$.


In [36]:
%%ampl_eval
reset;

param n;  # número de tipos de itens
param cp; # capacidade de peso de cada caixa
param r;  # número de lojas (rotas)

suffix master IN, binary;
suffix block IN, integer;

set I = 1..n ordered; # tipos de itens
set K = 1..r ordered; # lojas (rotas)
param w {I,K} >= 0; # peso do item tipo i para a loja k
param v {I,K} >= 0; # volume do item tipo i para a loja k
param inner{I,K} >= 0; # múltiplo da quantidade de itens a serem selecionados
param d {I,K} >= 0; # demanda de itens tipo i para a loja k

param cv {K} >= 0; # capacidade volumétrica de cada loja (rota)
param np {K} >= 0; # capacidade em número de peças para cada loja (rota)
param T {I,K} >= 0; # tempo de espera no backlog de produção
param Vr {K} >= 0; # volume da rota da chave k

param capac_vol_cx := min {k in K} cv [k];

param maxVal_vol := sum {i in I, k in K} d [i,k] *v [i,k];

param maxbins := ceil(maxVal_vol / capac_vol_cx);

set J = 1..maxbins ordered;

var e {I,J,K} integer >= 0; # número de itens tipo i na caixa j para a chave loja k
var y {J,K} binary; # 1 se a caixa j é usada para a loja k

minimize FO:  sum {i in I, k in K, j in J} y[j,k];

s.t. R1_capacidade_mochila_peso {j in J, k in K}:
   sum {i in I} w[i,k] * e[i,j,k] * inner[i,k] <= cp * y[j,k] suffix block j;

s.t. R2_capacidade_mochila_volume {j in J, k in K}:
   sum {i in I} v[i,k] * e[i,j,k] * inner[i,k] <= cv[k] * y[j,k] suffix block j;

s.t. R3_capacidade_mochila_pecas {j in J, k in K}:
   sum {i in I} e[i,j,k] * inner[i,k] <= np[k] * y[j,k] suffix block j;

s.t. R4_quantidade_demanda {i in I, k in K}:
   sum {j in J} e[i,j,k]* inner[i,k] = d[i,k] suffix master 1;

# Limite superior para o somatório dos volumes das caixas em uma rota
s.t. R5_volume_rota_superior {k in K}:
    sum {j in J} y[j,k] * cv[k] <= Vr[k];

### 2.1.1- Importar dados

In [37]:
df_process_situacao2 = df_process_capacitado.query('chave_loja_buffer_onda in ["28_EN_4","10_CX_1","3_CX_9"]')
#df_process_capacitado.query(f"chave_loja_buffer_onda in ({lojas_selecionadas_maiores})")

df_process_situacao2

,Id,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,...,pai_cor,chave_loja_buffer_onda,Volume_unit,tempo_espera_dias,prioridade,volume_util_caixa_Caixa_1,volume_util_caixa_Caixa_2,volume_util_caixa_Caixa_3,volume_util_padrao,Peças_max
14,14,1429,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
15,15,1499,2023-10-24,534996762,364,167,90,0.090,1,CARTON_M,...,777_1.0,10_CX_1,5470920,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
16,16,1499,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
17,17,909,2023-10-24,539354355,250,100,70,0.200,2,CARTON_M,...,777_95.0,10_CX_1,1750000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
18,18,1499,2023-10-24,539354347,250,100,55,0.180,1,CARTON_M,...,777_95.0,10_CX_1,1375000,29,78,27778950.0,83393894.4,75538008.0,83393894.4,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61584,61584,8067,2023-11-22,878451301,431,234,19,0.215,24,CARTON_M,...,930_283.0,28_EN_4,1916226,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61585,61585,9984,2023-11-22,878451328,424,245,18,0.187,15,CARTON_M,...,930_283.0,28_EN_4,1869840,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61586,61586,1157,2023-11-22,873049031,450,306,23,0.287,2,CARTON_M,...,943_18.0,28_EN_4,3167100,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512
61587,61587,8666,2023-11-22,873049102,450,315,20,0.307,2,CARTON_M,...,943_18.0,28_EN_4,2835000,0,78,27778950.0,83393894.4,75538008.0,75538008.0,512


In [38]:
df_process_situacao2['Peças_max'].unique()

array([170, 512], dtype=int64)

In [39]:
df_process_situacao2.columns

Index(['Id', 'Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade_rota', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'tempo_espera_dias',
       'prioridade', 'volume_util_caixa_Caixa_1', 'volume_util_caixa_Caixa_2',
       'volume_util_caixa_Caixa_3', 'volume_util_padrao', 'Peças_max'],
      dtype='object')

### 2.1.2- Entrada de dados no modelo

In [40]:
# Adicionar uma coluna para identificar cada item dentro de cada chave_loja_buffer_onda
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peso_unit'

df_matriz_peso = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_peso.columns.name = None

df_matriz_peso

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,0.090,0.303,0.220
1,2,0.090,0.323,0.153
2,3,0.200,0.295,0.230
3,4,0.200,0.310,0.230
4,5,0.180,0.328,0.187
...,...,...,...,...
402,403,0.143,0.215,0.000
403,404,0.160,0.187,0.000
404,405,0.000,0.287,0.000
405,406,0.000,0.307,0.000


In [41]:
# Variável avaliada:
variavel_ik = 'Volume_unit'

df_matriz_volume = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_volume.columns.name = None

df_matriz_volume

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,5470920.0,3303008.0,1556100.0
1,2,5470920.0,3553770.0,1681500.0
2,3,1750000.0,3297150.0,1430100.0
3,4,1750000.0,3428352.0,2275200.0
4,5,1375000.0,3583140.0,2096496.0
...,...,...,...,...
402,403,3961375.0,1916226.0,0.0
403,404,3970560.0,1869840.0,0.0
404,405,0.0,3167100.0,0.0
405,406,0.0,2835000.0,0.0


In [42]:
# volume total da demanda
df_matriz_volume['10_CX_1'].sum()

731276417.0

In [43]:
# volume total da demanda
df_matriz_volume['10_CX_1'].sum()>4.000000e+10

False

In [44]:
# volume total da demanda
df_matriz_volume['28_EN_4'].sum()


1949446242.0

In [45]:
# volume total da demanda
df_matriz_volume['28_EN_4'].sum()>2.720000e+10


False

In [46]:
# volume total da demanda
df_matriz_volume['3_CX_9'].sum()


580453446.0

In [47]:
# volume total da demanda
df_matriz_volume['3_CX_9'].sum()>2.720000e+10


False

In [48]:
# Variável avaliada:
variavel_ik = 'tempo_espera_dias'

df_matriz_tempo = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_tempo.columns.name = None

df_matriz_tempo

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,29.0,23.0,19.0
1,2,29.0,23.0,19.0
2,3,29.0,23.0,19.0
3,4,29.0,23.0,19.0
4,5,29.0,23.0,19.0
...,...,...,...,...
402,403,6.0,0.0,0.0
403,404,6.0,0.0,0.0
404,405,0.0,0.0,0.0
405,406,0.0,0.0,0.0


In [49]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Inner'

df_matriz_inner = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_inner.columns.name = None

df_matriz_inner

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
402,403,1.0,1.0,0.0
403,404,1.0,1.0,0.0
404,405,0.0,1.0,0.0
405,406,0.0,1.0,0.0


In [50]:
# Adicionar uma coluna para identificar cada item dentro de cada loja
df_process_situacao2['item_loja'] = df_process_situacao2.groupby('chave_loja_buffer_onda').cumcount() + 1

# Variável avaliada:
variavel_ik = 'Peças'

df_matriz_pecas = df_process_situacao2.pivot_table(index='item_loja', columns='chave_loja_buffer_onda', values= variavel_ik, fill_value=0).reset_index()

df_matriz_pecas.columns.name = None

df_matriz_pecas

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,10.0,7.0
1,2,1.0,10.0,6.0
2,3,2.0,6.0,17.0
3,4,2.0,12.0,4.0
4,5,1.0,5.0,12.0
...,...,...,...,...
402,403,9.0,24.0,0.0
403,404,6.0,15.0,0.0
404,405,0.0,2.0,0.0
405,406,0.0,2.0,0.0


Capacidade da caixa como variável

In [51]:
# Variável avaliada:
variavel_ik = 'Peças_max'

df_matriz_caixa_capacidade_pecas = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_pecas.columns.name = None

df_matriz_caixa_capacidade_pecas

,chave_loja_buffer_onda,Peças_max
0,10_CX_1,170.0
1,28_EN_4,512.0
2,3_CX_9,170.0


In [52]:
# Variável avaliada:
variavel_ik = 'volume_util_padrao'

df_matriz_caixa_capacidade_volume = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_volume.columns.name = None

df_matriz_caixa_capacidade_volume

,chave_loja_buffer_onda,volume_util_padrao
0,10_CX_1,83393894.4
1,28_EN_4,75538008.0
2,3_CX_9,83393894.4


In [53]:
# Variável avaliada:
variavel_ik = 'Capacidade_rota'

df_matriz_caixa_capacidade_rota = df_process_situacao2.pivot_table(index='chave_loja_buffer_onda',  values= variavel_ik, fill_value=0).reset_index()

df_matriz_caixa_capacidade_rota.columns.name = None

df_matriz_caixa_capacidade_rota

,chave_loja_buffer_onda,Capacidade_rota
0,10_CX_1,4.000000e+10
1,28_EN_4,2.720000e+10
2,3_CX_9,2.720000e+10


Características das peças

In [54]:
peso =  df_matriz_peso
pecas = df_matriz_pecas  #demanda
volume = df_matriz_volume
inner = df_matriz_inner
t = df_matriz_tempo

Características da caixa

In [55]:
cp = 23
cv = df_matriz_caixa_capacidade_volume
np = df_matriz_caixa_capacidade_pecas

#### Parâmetros

In [56]:
cv['volume_util_padrao'].values

array([83393894.39999999, 75538008.        , 83393894.39999999])

In [57]:
ampl.param["n"] = len(pecas)   # tamanho da loja que tem mais itens

ampl.param["cp"] = cp
ampl.param["r"] = len(volume.columns) -1  # qtde de lojas


v = volume.drop(columns='item_loja')
w = peso.drop(columns='item_loja')
d= pecas.drop(columns='item_loja')
inner_ = inner.drop(columns='item_loja')
t_ = t.drop(columns='item_loja')
vol_rota = df_matriz_caixa_capacidade_rota.drop(columns='chave_loja_buffer_onda')

ampl.param["w"] = {(i + 1, k + 1): w.iloc[i, k] for i in range(len(w)) for k in range(len(w.columns))}
ampl.param["T"] = {(i + 1, k + 1): t_.iloc[i, k] for i in range(len(t_)) for k in range(len(t_.columns))}
ampl.param["v"] = {(i + 1, k + 1): v.iloc[i, k] for i in range(len(v)) for k in range(len(v.columns))}
ampl.param["d"] = {(i + 1, k + 1): d.iloc[i, k] for i in range(len(d)) for k in range(len(d.columns))}
ampl.param["inner"] = {(i + 1, k + 1): inner_.iloc[i, k] for i in range(len(inner_)) for k in range(len(inner_.columns))}

ampl.param["cv"] = cv['volume_util_padrao'].values
ampl.param["np"] = np['Peças_max'].values
ampl.param["Vr"] = {(k): vol_rota.iloc[k - 1] for k in range(1, len(vol_rota)+1)}

In [58]:
ampl.getValue('maxVal_vol')

23928776675.0

In [59]:
%%ampl_eval
display Vr;

Vr [*] :=
1  4e+10
2      2.72e+10
3      2.72e+10
;



In [60]:
ampl.getValue('maxbins')

317

### 2.1.3- Solvers

#### Solver Highs

In [61]:
ampl.option["solver"] = "highs"
ampl.option["highs_options"] = "outlev=2 timelimit=300 mip_rel_gap=1e-02"
ampl.solve()

HiGHS 1.7.0:   tech:outlev = 1
  lim:time = 300
  mip:relgaptol = 0.01
Running HiGHS 1.7.0 (git hash: 50670fd): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [3e-02, 8e+07]
  Cost   [4e+02, 4e+02]
  Bound  [1e+00, 3e+01]
  RHS    [1e+00, 9e+01]
Presolving model
3835 rows, 312245 cols, 1248029 nonzeros  0s
3835 rows, 312245 cols, 1248029 nonzeros  8s
Objective function is integral with scale 0.002457

Solving MIP model with:
   3835 rows
   312245 cols (73861 binary, 238384 integer, 0 implied int., 0 continuous)
   1248029 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     8.7s
         0       0         0   0.00%   125099.346906   inf     

In [62]:
%%ampl_eval
display d, w, v, inner,T;

:       d      w        v     inner   T     :=
1   1    1   0.09     5470920    1    29
1   2   10   0.303    3303010    1    23
1   3    7   0.22     1556100    1    19
2   1    1   0.09     5470920    1    29
2   2   10   0.323    3553770    1    23
2   3    6   0.153    1681500    1    19
3   1    2   0.2      1750000    1    29
3   2    6   0.295    3297150    1    23
3   3   17   0.23     1430100    1    19
4   1    2   0.2      1750000    1    29
4   2   12   0.31     3428350    1    23
4   3    4   0.23     2275200    1    19
5   1    1   0.18     1375000    1    29
5   2    5   0.328    3583140    1    23
5   3   12   0.187    2096500    1    19
6   1    2   0.18     1375000    1    29
6   2   18   0.303    2701220    1    23
6   3    2   0.18     1772820    1    19
7   1    2   0.2      1350000    1    29
7   2    2   0.44     3200700    1    23
7   3    1   0.153    1681500    1    19
8   1    6   0.1      1412220    1    19
8   2    1   0.47     3426520    1    23
8   3    1

In [63]:
%%ampl_eval
display e, y;

e [*,*,1]
:     1   2   3   4   5   6   7   8   9  10   11  12   13  14  15       16   :=
1     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
2     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
3     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
4     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
5     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
6     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
7     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
8     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
9     0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
10    0   0   0   0   0   0   0   0   0   0    0   0    0   0   0    0
11    0   0   0   0   0   0   0   0   0   0    0   0    2   0   0    0
12    0   0   0   0   0   0   0   0   0   0    0   0    1   0   0    0
13    0   0   0   0   0   0   0   0   0   0    0   0    0 

In [64]:
inner


,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,1.0,1.0
1,2,1.0,1.0,1.0
2,3,1.0,1.0,1.0
3,4,1.0,1.0,1.0
4,5,1.0,1.0,1.0
...,...,...,...,...
402,403,1.0,1.0,0.0
403,404,1.0,1.0,0.0
404,405,0.0,1.0,0.0
405,406,0.0,1.0,0.0


In [65]:
e_s2_highs = ampl.getVariable('e').getValues().toPandas()
e_s2_highs = e_s2_highs.reset_index()
e_s2_highs.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_highs = e_s2_highs.query('valor_multiplo >0')
e_s2_highs#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
58,1,20,2,4.0
287,1,96,3,2.0
476,1,159,3,4.0
665,1,222,3,1.0
892,1,298,2,6.0
...,...,...,...,...
385444,406,97,2,2.0
386422,407,106,2,5.0
386437,407,111,2,3.0
386620,407,172,2,2.0


In [66]:
df_loja_10_CX_1= e_s2_highs.query('chave_loja_buffer_onda == 1 & valor_multiplo>0').merge(inner[['item_loja','10_CX_1']], how='inner', on='item_loja')
df_loja_10_CX_1['pecas_selecionadas'] = df_loja_10_CX_1['valor_multiplo'] * df_loja_10_CX_1['10_CX_1']
df_loja_10_CX_1

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo,10_CX_1,pecas_selecionadas
0,1,310,1,1.0,1.0,1.0
1,2,260,1,1.0,1.0,1.0
2,3,260,1,2.0,1.0,2.0
3,4,41,1,2.0,1.0,2.0
4,5,114,1,1.0,1.0,1.0
...,...,...,...,...,...,...
510,402,293,1,1.0,1.0,1.0
511,403,68,1,2.0,1.0,2.0
512,403,193,1,7.0,1.0,7.0
513,404,196,1,1.0,1.0,1.0


In [67]:
df_loja_10_CX_1['pecas_selecionadas'].sum()

1948.0

In [68]:
df_loja_28_EN_4= e_s2_highs.query('chave_loja_buffer_onda == 2 & valor_multiplo>0').merge(inner[['item_loja','28_EN_4']], how='inner', on='item_loja')
df_loja_28_EN_4['pecas_selecionadas'] = df_loja_28_EN_4['valor_multiplo'] * df_loja_28_EN_4['28_EN_4']
df_loja_28_EN_4

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo,28_EN_4,pecas_selecionadas
0,1,20,2,4.0,1.0,4.0
1,1,298,2,6.0,1.0,6.0
2,2,275,2,8.0,1.0,8.0
3,2,288,2,2.0,1.0,2.0
4,3,29,2,6.0,1.0,6.0
...,...,...,...,...,...,...
1049,406,97,2,2.0,1.0,2.0
1050,407,106,2,5.0,1.0,5.0
1051,407,111,2,3.0,1.0,3.0
1052,407,172,2,2.0,1.0,2.0


In [69]:
df_loja_28_EN_4['pecas_selecionadas'].sum()

3187.0

In [70]:
df_loja_3_CX_9= e_s2_highs.query('chave_loja_buffer_onda == 3 & valor_multiplo>0').merge(inner[['item_loja','3_CX_9']], how='inner', on='item_loja')
df_loja_3_CX_9['pecas_selecionadas'] = df_loja_3_CX_9['valor_multiplo'] * df_loja_3_CX_9['3_CX_9']
df_loja_3_CX_9

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo,3_CX_9,pecas_selecionadas
0,1,96,3,2.0,1.0,2.0
1,1,159,3,4.0,1.0,4.0
2,1,222,3,1.0,1.0,1.0
3,2,205,3,6.0,1.0,6.0
4,3,3,3,7.0,1.0,7.0
...,...,...,...,...,...,...
301,168,205,3,7.0,1.0,7.0
302,169,208,3,9.0,1.0,9.0
303,170,114,3,6.0,4.0,24.0
304,170,249,3,16.0,4.0,64.0


In [71]:
df_loja_3_CX_9.head(22)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo,3_CX_9,pecas_selecionadas
0,1,96,3,2.0,1.0,2.0
1,1,159,3,4.0,1.0,4.0
2,1,222,3,1.0,1.0,1.0
3,2,205,3,6.0,1.0,6.0
4,3,3,3,7.0,1.0,7.0
5,3,14,3,2.0,1.0,2.0
6,3,81,3,4.0,1.0,4.0
7,3,148,3,3.0,1.0,3.0
8,3,195,3,1.0,1.0,1.0
9,4,148,3,1.0,1.0,1.0


In [72]:
pecas.head(22)

,item_loja,10_CX_1,28_EN_4,3_CX_9
0,1,1.0,10.0,7.0
1,2,1.0,10.0,6.0
2,3,2.0,6.0,17.0
3,4,2.0,12.0,4.0
4,5,1.0,5.0,12.0
5,6,2.0,18.0,2.0
6,7,2.0,2.0,1.0
7,8,6.0,1.0,1.0
8,9,1.0,8.0,2.0
9,10,1.0,3.0,1.0


In [73]:
for col in w.columns:
    print(f'demanda loja "{col}": {pecas[col].sum()}\n')

demanda loja "10_CX_1": 1948.0

demanda loja "28_EN_4": 3187.0

demanda loja "3_CX_9": 1690.0



In [74]:
df_loja_3_CX_9['pecas_selecionadas'].sum()

1690.0

In [75]:
vetores_por_loja = {}

# Iterar sobre cada grupo (grupo de cada loja)
for nome_loja, grupo in df_process_situacao2.groupby('chave_loja_buffer_onda'):
    # Extrair os IDs do grupo como um vetor
    vetor_ids = grupo['Id'].tolist()
    # Adicionar o vetor à lista de vetores, usando o nome da loja como chave
    vetores_por_loja[nome_loja] = vetor_ids
    # Atribuir o vetor a uma variável com o nome da loja
    exec(f'vetor_{nome_loja} = vetor_ids')

In [76]:
# Lista dos caracteres finais dos nomes dos vetores
char_final = [col for col in w.columns[:3]]

# Criar o dicionário mapeando os caracteres finais aos vetores
vetores = {char: globals()[f'vetor_{char}'] for char in char_final}

In [77]:
df_agrupamento_loja1_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_highs_s2 = df_agrupamento_loja1_highs_s2.reset_index()
df_agrupamento_loja1_highs_s2.columns.name = None
df_agrupamento_loja1_highs_s2 # as colunas são as caixas

,item_loja,11,13,16,18,19,25,29,31,38,...,235,244,245,260,261,277,284,293,304,310
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401,402,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
402,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
df_agrupamento_loja1_highs_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_highs_s2

,item_loja,11,13,16,18,19,25,29,31,38,...,245,260,261,277,284,293,304,310,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14,10_CX_1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15,10_CX_1
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,16,10_CX_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,10_CX_1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,10_CX_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31851,10_CX_1
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31852,10_CX_1
401,402,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31853,10_CX_1
402,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31854,10_CX_1


In [79]:
df_agrupamento_loja2_highs_s2 = e_s2_highs.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_highs_s2 = df_agrupamento_loja2_highs_s2.reset_index()
df_agrupamento_loja2_highs_s2.columns.name = None
df_agrupamento_loja2_highs_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_highs_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_highs_s2

,item_loja,1,3,5,6,9,10,11,12,13,...,307,308,309,310,312,313,314,316,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33800,28_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33801,28_EN_4
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33802,28_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33803,28_EN_4
4,5,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33804,28_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,403,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61584,28_EN_4
403,404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61585,28_EN_4
404,405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61586,28_EN_4
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61587,28_EN_4


In [80]:
y_s2_highs = ampl.getVariable('y').getValues().toPandas()
y_s2_highs = y_s2_highs.reset_index()
y_s2_highs.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_highs = y_s2_highs.query('selecao>0')

df_caixas_por_loja_highs_s2 = y_s2_highs.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_highs_s2 = df_caixas_por_loja_highs_s2.reset_index()
df_caixas_por_loja_highs_s2.columns.name = None
df_caixas_por_loja_highs_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,0.0
1,3,0.0,1.0,1.0
2,5,0.0,1.0,0.0
3,6,0.0,1.0,0.0
4,9,0.0,1.0,0.0
...,...,...,...,...
240,310,1.0,1.0,1.0
241,312,0.0,1.0,0.0
242,313,0.0,1.0,0.0
243,314,0.0,1.0,1.0


In [81]:
df_caixas_por_loja_highs_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_highs_s2

,caixas,10_CX_1,28_EN_4,3_CX_9
0,1,0.0,1.0,0.0
1,3,0.0,1.0,1.0
2,5,0.0,1.0,0.0
3,6,0.0,1.0,0.0
4,9,0.0,1.0,0.0
...,...,...,...,...
240,310,1.0,1.0,1.0
241,312,0.0,1.0,0.0
242,313,0.0,1.0,0.0
243,314,0.0,1.0,1.0


In [82]:
print('Solver Highs')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_highs_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver Highs
número de caixas para a chave_loja_buffer_onda "10_CX_1": 44.0
número de caixas para a chave_loja_buffer_onda "28_EN_4": 219.0
número de caixas para a chave_loja_buffer_onda "3_CX_9": 50.0

total de caixas: 313.0


O número de caixas selecionadas foi 1 unidade acima do modelo de minimização do número de caixas

In [83]:
f

NameError: name 'f' is not defined

#### Solver SCIP

In [84]:
ampl.option["solver"] = "scip"
ampl.option["scip_options"] = "outlev=1 timelimit=300"
ampl.solve()

SCIP 9.0.0:   tech:outlev = 1
  lim:time = 300
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 229 upgd conss, 0 impls, 229 clqs
   (3.0s) probing: 51/73861 (0.1%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (3.0s) probing aborted: 50/50 successive totally useless probings
   (3.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (5.0s) symmetry computation finished: 204 generators found (max: 204, log10 of symmetry group size: inf) (symcode time: 2.00)
dynamic symmetry handling statistics:
   orbitopal reduction:        3 components: 317x7, 317x3, 317x3
   orbital reduction:          1 components of sizes 173
   lexicographic reduction:  194 permutations with support sizes 634, 344, 634, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344, 344

In [85]:
e_s2_scip = ampl.getVariable('e').getValues().toPandas()
e_s2_scip = e_s2_scip.reset_index()
e_s2_scip.columns = ['item_loja','caixa', 'chave_loja_buffer_onda', 'valor_multiplo']
e_s2_scip = e_s2_scip.query('valor_multiplo >0')
e_s2_scip#.head(20)

,item_loja,caixa,chave_loja_buffer_onda,valor_multiplo
58,1,20,2,4.0
287,1,96,3,2.0
476,1,159,3,4.0
665,1,222,3,1.0
892,1,298,2,6.0
...,...,...,...,...
385444,406,97,2,2.0
386422,407,106,2,5.0
386437,407,111,2,3.0
386620,407,172,2,2.0


In [86]:
df_agrupamento_loja1_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 1').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja1_scip_s2 = df_agrupamento_loja1_scip_s2.reset_index()
df_agrupamento_loja1_scip_s2.columns.name = None
df_agrupamento_loja1_scip_s2['Id'] = vetores[f"{w.columns[0]}"]
df_agrupamento_loja1_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[0]

df_agrupamento_loja1_scip_s2

,item_loja,11,13,16,18,19,25,29,31,38,...,245,260,261,277,284,293,304,310,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,14,10_CX_1
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,15,10_CX_1
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,16,10_CX_1
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,10_CX_1
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18,10_CX_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31851,10_CX_1
400,401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31852,10_CX_1
401,402,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31853,10_CX_1
402,403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31854,10_CX_1


In [87]:
df_agrupamento_loja2_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 2').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja2_scip_s2 = df_agrupamento_loja2_scip_s2.reset_index()
df_agrupamento_loja2_scip_s2.columns.name = None
df_agrupamento_loja2_scip_s2['Id'] = vetores[f"{w.columns[1]}"]
df_agrupamento_loja2_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[1]

df_agrupamento_loja2_scip_s2

,item_loja,1,3,5,6,9,10,11,12,13,...,307,308,309,310,312,313,314,316,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33800,28_EN_4
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33801,28_EN_4
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33802,28_EN_4
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33803,28_EN_4
4,5,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33804,28_EN_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,403,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61584,28_EN_4
403,404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61585,28_EN_4
404,405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61586,28_EN_4
405,406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61587,28_EN_4


In [88]:
df_agrupamento_loja3_scip_s2 = e_s2_scip.query('chave_loja_buffer_onda == 3').pivot_table(index='item_loja', columns='caixa', values='valor_multiplo', fill_value=0)
df_agrupamento_loja3_scip_s2 = df_agrupamento_loja3_scip_s2.reset_index()
df_agrupamento_loja3_scip_s2.columns.name = None
df_agrupamento_loja3_scip_s2['Id'] = vetores[f"{w.columns[2]}"]
df_agrupamento_loja3_scip_s2['chave_loja_buffer_onda'] = list(w.columns)[2]

df_agrupamento_loja3_scip_s2

,item_loja,3,10,12,14,15,32,40,54,63,...,259,279,280,282,308,310,314,316,Id,chave_loja_buffer_onda
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,652,3_CX_9
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,653,3_CX_9
2,3,7.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,654,3_CX_9
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,655,3_CX_9
4,5,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,656,3_CX_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32838,3_CX_9
167,168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32839,3_CX_9
168,169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32840,3_CX_9
169,170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32841,3_CX_9


In [89]:
y_s2_scip = ampl.getVariable('y').getValues().toPandas()
y_s2_scip = y_s2_scip.reset_index()
y_s2_scip.columns = ['Caixa','chave_loja_buffer_onda', 'selecao']
y_s2_scip = y_s2_scip.query('selecao>0')

df_caixas_por_loja_scip_s2 = y_s2_scip.pivot_table(index='Caixa', columns='chave_loja_buffer_onda', values='selecao', fill_value=0)
df_caixas_por_loja_scip_s2 = df_caixas_por_loja_scip_s2.reset_index()
df_caixas_por_loja_scip_s2.columns.name = None
df_caixas_por_loja_scip_s2 # as colunas são as lojas

,Caixa,1,2,3
0,1,0.0,1.0,0.0
1,3,0.0,1.0,1.0
2,5,0.0,1.0,0.0
3,6,0.0,1.0,0.0
4,9,0.0,1.0,0.0
...,...,...,...,...
240,310,1.0,1.0,1.0
241,312,0.0,1.0,0.0
242,313,0.0,1.0,0.0
243,314,0.0,1.0,1.0


In [90]:
df_caixas_por_loja_scip_s2.columns = (['caixas'] + list(w.columns)) # os número das colunas são os nomes das lojas
df_caixas_por_loja_scip_s2

,caixas,10_CX_1,28_EN_4,3_CX_9
0,1,0.0,1.0,0.0
1,3,0.0,1.0,1.0
2,5,0.0,1.0,0.0
3,6,0.0,1.0,0.0
4,9,0.0,1.0,0.0
...,...,...,...,...
240,310,1.0,1.0,1.0
241,312,0.0,1.0,0.0
242,313,0.0,1.0,0.0
243,314,0.0,1.0,1.0


In [91]:
print('Solver SCIP')
total_caixas = 0
for col in w.columns:
    print(f'número de caixas para a chave_loja_buffer_onda "{col}": {df_caixas_por_loja_scip_s2[col].sum()}')
    total_caixas += df_caixas_por_loja_highs_s2[col].sum()
print(f'\ntotal de caixas: {total_caixas}')

Solver SCIP
número de caixas para a chave_loja_buffer_onda "10_CX_1": 44.0
número de caixas para a chave_loja_buffer_onda "28_EN_4": 219.0
número de caixas para a chave_loja_buffer_onda "3_CX_9": 50.0

total de caixas: 313.0
